In [5]:
import time
import random
from random import randint
from mancala import Mancala
from copy import deepcopy
random.seed(140)

In [6]:
class Mancala:
    def __init__(self, pits_per_player=6, stones_per_pit = 4):
        """
        The constructor for the Mancala class defines several instance variables:

        pits_per_player: This variable stores the number of pits each player has.
        stones_per_pit: It represents the number of stones each pit contains at the start of any game.
        board: This data structure is responsible for managing the Mancala board.
        current_player: This variable takes the value 1 or 2, as it's a two-player game, indicating which player's turn it is.
        moves: This is a list used to store the moves made by each player. It's structured in the format (current_player, chosen_pit).
        p1_pits_index: A list containing two elements representing the start and end indices of player 1's pits in the board data structure.
        p2_pits_index: Similar to p1_pits_index, it contains the start and end indices for player 2's pits on the board.
        p1_mancala_index and p2_mancala_index: These variables hold the indices of the Mancala pits on the board for players 1 and 2, respectively.
        """
        self.pits_per_player = pits_per_player
        self.board = [stones_per_pit] * ((pits_per_player+1) * 2)  # Initialize each pit with stones_per_pit number of stones 
        self.players = 2
        self.current_player = 1
        self.moves = []
        self.p1_pits_index = [0, self.pits_per_player-1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player+1, len(self.board)-1-1]
        self.p2_mancala_index = len(self.board)-1
        
        # Zeroing the Mancala for both players
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0

    def display_board(self):
        """
        Displays the board in a user-friendly format
        """
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        player_2_mancala = self.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            else:    
                print('{} -> | {} | {} | <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            
        print('         {}         '.format(player_1_mancala))
        turn = 'P1' if self.current_player == 1 else 'P2'
        print('Turn: ' + turn)
        
    def valid_move(self, pit):
        """
        Function to check if the pit chosen by the current_player is a valid move.
        """
        check = False
        
        # write your code here
        # if the pit is out of range or empty
        if(self.current_player == 1):
            if(pit <= self.pits_per_player):
                if(self.board[pit - 1] != 0):
                    check = True
        else:
            if(pit <= self.pits_per_player):
                if(self.board[pit + self.pits_per_player] != 0):
                    check = True
                    
        return check
        
    def random_move_generator(self):
        """
        Function to generate random valid moves with non-empty pits for the random player
        """
        # write your code here
        valid = False
        
        # generating a random valid move
        while(valid == False):
            move = randint(1, self.pits_per_player)
            valid = self.valid_move(move)
            
        return move
    
    def play(self, pit):
        """
        This function simulates a single move made by a specific player using their selected pit. It primarily performs three tasks:
        1. It checks if the chosen pit is a valid move for the current player. If not, it prints "INVALID MOVE" and takes no action.
        2. It verifies if the game board has already reached a winning state. If so, it prints "GAME OVER" and takes no further action.
        3. After passing the above two checks, it proceeds to distribute the stones according to the specified Mancala rules.

        Finally, the function then switches the current player, allowing the other player to take their turn.
        """
        # write your code here
        # print("Player", self.current_player, "chose pit:", pit)
        # checking if pit is valid
        test = self.valid_move(pit)
        if(test != True):
            print("INVALID MOVE")
            print()
            return self.board
        
        # print()
    
        # checking for winning state
        win = self.winning_eval()
        if(win == (True, 1) or win == (True, 2) or win == (True, -1)):
            # print("GAME OVER")
            return self.board
        
        # determining current player
        if(self.current_player == 1):
            self.moves.append((1, pit))
            moves = self.board[pit - 1]
            self.board[pit - 1] = 0 # adjusting current count
            for i in range(1, moves + 1):
                nextpit = (pit + i - 1) % (len(self.board))
                # make sure that pit is not opponent's mancala
                if(nextpit == self.p2_mancala_index):
                    nextpit = (nextpit + 1) % (len(self.board))
                    
                # if on last move
                if(i == (moves)): 
                    if(self.board[nextpit] == 0): # the next pit is empty
                        # check if it is our side but not our mancala
                        if(nextpit in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1)):
                            # print("Capture!")
                            opp_pit = 2*self.pits_per_player - nextpit
                            extras = self.board[opp_pit] + 1 # taking the current marble as well
                            self.board[opp_pit] = 0
                            # place these in my mancala
                            self.board[self.p1_mancala_index] += extras
                            break

                self.board[nextpit] += 1 # depositing a marble
                        
            # switching to player 2
            self.current_player = 2
        else:
            self.moves.append((2, pit))
            # moving base pit up accordingly to be at the right index
            pit = pit + self.pits_per_player + 1
            moves = self.board[pit - 1]
            self.board[pit - 1] = 0 # adjusting current count
            for i in range(1, moves + 1):
                nextpit = (pit + i - 1) % (len(self.board))
                # make sure that pit is not opponent's mancala
                if(nextpit == self.p1_mancala_index):
                    nextpit = (nextpit + 1) % (len(self.board))
                # if on last move
                if(i == (moves)): 
                    if(self.board[nextpit] == 0): # the next pit is empty
                        # check if it is our side but not our mancala
                        if(nextpit in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1)):
                            # print("Capture!")
                            opp_pit = 2*self.pits_per_player - nextpit
                            extras = self.board[opp_pit] + 1 # taking the current marble as well
                            self.board[opp_pit] = 0
                            # place these in my mancala
                            self.board[self.p2_mancala_index] += extras
                            break
                            
                self.board[nextpit] += 1 # deposting a marble
                        
            # switching to player 1
            self.current_player = 1
                   
        return self.board
    
    def winning_eval(self):
        """
        Function to verify if the game board has reached the winning state.
        Hint: If either of the players' pits are all empty, then it is considered a winning state.
        """
        # if a player's side is empty, the game is over
        # the player who still has stones on their side of the board places these into their mancala
        # whoever has the most stones in their mancala wins
        state = True
        pl = -1
        
        for i in range(self.pits_per_player): # player one's side is empty
            if (self.board[i] != 0):
                state = False
                
        if(state == True):
            # count the number of stones on player two's side, adding these to their mancala
            for i in range(self.pits_per_player):
                stones = self.board[self.pits_per_player + 1 + i]
                self.board[self.pits_per_player + 1 + i] = 0 # taking them out of the pits
                self.board[self.p2_mancala_index] = self.board[self.p2_mancala_index] + stones
                
            # determining who won
            if (self.board[self.p1_mancala_index] > self.board[self.p2_mancala_index]):
                # player one wins
                return (state, 1)
            elif (self.board[self.p2_mancala_index] > self.board[self.p1_mancala_index]):
                # player two wins
                return (state, 2)
            else:
                # tie
                return (state, 0)
    
        state = True
        for i in range(self.pits_per_player): # player two's side is empty
            if (self.board[self.pits_per_player + 1 + i] != 0):
                state = False
                
        if (state == True):
            # count the number of stones on player one's side, adding these to their mancala
            for i in range(self.pits_per_player):
                stones = self.board[i]
                self.board[i] = 0
                self.board[self.p1_mancala_index] = self.board[self.p1_mancala_index] + stones
                
            # determining who won
            if (self.board[self.p1_mancala_index] > self.board[self.p2_mancala_index]):
                # player one wins
                return (state, 1)
            elif (self.board[self.p2_mancala_index] > self.board[self.p1_mancala_index]):
                # player two wins
                return (state, 2)
            else:
                # tie
                return (state, 0)
        
        return (state, pl)

    def utility(self):
        return self.board[self.p1_mancala_index] - self.board[self.p2_mancala_index]
    
    def minimax(self, depth, max_depth, maximizing_player):
        if depth == max_depth or self.winning_eval()[0]:
            return self.utility(), None

        if maximizing_player:  # Player 1 (Max)
            best_value = float('-inf')
            best_move = None
            for pit in range(1, self.pits_per_player + 1):
                if self.valid_move(pit):
                    game_copy = deepcopy(self)
                    game_copy.play(pit)
                    value, _ = game_copy.minimax(depth + 1, max_depth, False)
                    if value > best_value:
                        best_value = value
                        best_move = pit
            return best_value, best_move
        else:  # Player 2 (Min)
            best_value = float('inf')
            best_move = None
            for pit in range(1, self.pits_per_player + 1):
                if self.valid_move(pit):
                    game_copy = deepcopy(self)
                    game_copy.play(pit)
                    value, _ = game_copy.minimax(depth + 1, max_depth, True)
                    if value < best_value:
                        best_value = value
                        best_move = pit
            return best_value, best_move

    def get_best_move(self, max_depth):
        _, move = self.minimax(0, max_depth, self.current_player == 1)
        return move 
    
    def alpha_beta(self, depth, alpha, beta, max_depth, maximizing_player):
        if depth == max_depth or self.winning_eval()[0]:
            return self.utility(), None

        if maximizing_player:  # Player 1 (Max)
            best_value = float('-inf')
            best_move = None
            for pit in range(1, self.pits_per_player + 1):
                if self.valid_move(pit):
                    game_copy = deepcopy(self)
                    game_copy.play(pit)
                    value, _ = game_copy.alpha_beta(depth + 1, alpha, beta, max_depth, False)
                    if value > best_value:
                        best_value = value
                        best_move = pit
                    alpha = max(alpha, best_value)
                    
                    # alpha beta pruning
                    if beta <= alpha:
                        break
                        
            return best_value, best_move
        else:  # Player 2 (Min)
            best_value = float('inf')
            best_move = None
            for pit in range(1, self.pits_per_player + 1):
                if self.valid_move(pit):
                    game_copy = deepcopy(self)
                    game_copy.play(pit)
                    value, _ = game_copy.alpha_beta(depth + 1, alpha, beta, max_depth, True)
                    if value < best_value:
                        best_value = value
                        best_move = pit
                    beta = min(beta, best_value)
                    
                    # alpha beta pruning
                    if beta <= alpha:
                        break
            return best_value, best_move
        
        
    def alpha_beta_move(self, max_depth):
        _, move = self.alpha_beta(0, float('-inf'), float('inf'), max_depth, self.current_player == 1)
        return move

### Random vs. Random

In [7]:
# Simulating 100 Random vs Random player Mancala games
p1counter = 0 
p2counter = 0 
tiecounter = 0 

# move counters
p1moves = 0
p2moves = 0

n = 100

for i in range(n):
    game = Mancala() # creating a new game every time
    
    # play until someone wins
    win = (False, -1)
    
    while(win[0] == False):  
        # determining player
        player = game.current_player
        
        if(player == 1):
            p1moves = p1moves + 1
        else:
            p2moves = p2moves + 1
            
            
        game.play(game.random_move_generator()) 
        # print()
        # game.display_board()
        
        # check for win
        win = game.winning_eval()
        
    # someone won - determine who
    if (win[1] == 1):
        p1counter = p1counter + 1
    elif (win[1] == 2):
        p2counter = p2counter + 1
    elif (win[1] == 0):
        tiecounter = tiecounter + 1
        
    del game # removing the game for the next round
        
    # print()
    # print()
    
print("Player 1 won " + str(p1counter/n * 100) + "% of the time.")
print("Player 2 won " + str(p2counter/n * 100) + "% of the time.")
print("Ties occurred " + str(tiecounter/n * 100) + "% of the time.")
print()

p1avgmoves = p1moves/n
p2avgmoves = p2moves/n

print("Player 1 won with an average of " + str(p1avgmoves) + " moves per game.")
print("Player 2 won with an average of " + str(p2avgmoves) + " moves per game.")

Player 1 won 50.0% of the time.
Player 2 won 47.0% of the time.
Ties occurred 3.0% of the time.

Player 1 won with an average of 20.79 moves per game.
Player 2 won with an average of 20.33 moves per game.


On average, each player wins about 45% of the time, plus or minus about 4%, depending on the percentage of ties. The average of moves per game is around 20 for both player one and player two. Also, despite this being two players choosing randomly, we can see a slight player one advantage.

### AI vs. Random Using Minimax

In [8]:
# Simulating 100 games of AI with Minimax vs Random player
ai_wins = 0  
random_wins = 0  
ties = 0

# Move counters
ai_moves = 0
random_moves = 0

max_depth = 5  # Number of plies

n = 100

for i in range(n):
    game = Mancala()
    while(game.winning_eval() == (False, -1)):
        player = game.current_player
        if player == 1:  # AI minimax player
            move = game.get_best_move(max_depth)
            ai_moves += 1
        else:  # Random player
            move = game.random_move_generator()
            random_moves += 1
        game.play(move)
        # game.display_board()
        # print()

    win = game.winning_eval()
    
    if (win == (True, 1)):
        ai_wins += 1
    elif (win == (True, 2)):
        random_wins += 1
    else:
        ties += 1
        
    del game # removing the game for the next round
        
    # print()
    # print()
    
ai_wins = ai_wins/n * 100
random_wins = random_wins/n * 100
ties = ties/n * 100

print(f"AI (Player 1) won {ai_wins}% of the time.")
print(f"Random (Player 2) won {random_wins}% of the time.")
print(f"Ties occurred {ties}% of the time.")
print()

p1avgmoves = ai_moves/n
p2avgmoves = random_moves/n

print("Player 1 won with an average of " + str(p1avgmoves) + " moves per game.")
print("Player 2 won with an average of " + str(p2avgmoves) + " moves per game.")

AI (Player 1) won 94.0% of the time.
Random (Player 2) won 4.0% of the time.
Ties occurred 2.0% of the time.

Player 1 won with an average of 14.47 moves per game.
Player 2 won with an average of 14.1 moves per game.


The AI Player wins over 70% of the time, and both player's average moves are around 14 moves per game. This can be expected since although our utility function is not great, especially for deeper plies, it is much easier for player 1 to win when player 2 is making random moves at every step, potentially making moves that cost them the game. 

The AI is better than random chance because it is using the minimax algorithm, wherein it is always maximizing its own move and minimizing the opponent's move. This is better than random chance since it is making an informed decision about what move it should make next. Random chance would give us an even split across all possible moves. However, the minimax algorithm allows us to make one specific, informed move, giving us a better move than random chance. Also, minimax "looks ahead", so it chooses the best move in the long run rather than the best move at that point in time, which is another reason why it is better than random chance.

### AI Player Using Alpha-Beta

In [9]:
# Simulating 100 games of AI with AB pruning vs Random player
ai_wins = 0  
random_wins = 0  
ties = 0

# Move counters
ai_moves = 0
random_moves = 0

max_depth = 5  # Number of plies

start_time = time.perf_counter()

for i in range(100):
    game = Mancala()
    while(game.winning_eval() == (False, -1)):
        player = game.current_player
        if player == 1:  # AI alpha-beta player
            move = game.alpha_beta_move(max_depth)
            ai_moves += 1
        else:  # Random player
            move = game.random_move_generator()
            random_moves += 1
        game.play(move)
        # game.display_board()
        # print()

    win = game.winning_eval()
    
    if (win == (True, 1)):
        ai_wins += 1
    elif (win == (True, 2)):
        random_wins += 1
    else:
        ties += 1
        
    del game # removing the game for the next round
        
    # print()
    # print()
    
end_time = time.perf_counter()
tot_elapsed_time = end_time - start_time
avg_elapsed_time = tot_elapsed_time/100

print(f"The average amount of time per game is {avg_elapsed_time:.4f} seconds.")
print()

print(f"AI (Player 1) won {ai_wins}% of the time.")
print(f"Random (Player 2) won {random_wins}% of the time.")
print(f"Ties occurred {ties}% of the time.")
print()

p1avgmoves = ai_moves/100
p2avgmoves = random_moves/100

print("Player 1 won with an average of " + str(p1avgmoves) + " moves per game.")
print("Player 2 won with an average of " + str(p2avgmoves) + " moves per game.")


The average amount of time per game is 0.3431 seconds.

AI (Player 1) won 94% of the time.
Random (Player 2) won 4% of the time.
Ties occurred 2% of the time.

Player 1 won with an average of 14.32 moves per game.
Player 2 won with an average of 13.9 moves per game.


These results are the same as the minimax AI player. This is because they are both reliant on the minimax algorithm to choose the best possible move for the AI player. The only thing that is different is that the alpha-beta algorithm uses pruning so that it doesn't have to look as far in the minimax tree, thereby shortening the computing time, but producing the same result.

### AI with Alpha Beta and Improved Utility

In [10]:
class Mancala:
    def __init__(self, pits_per_player=6, stones_per_pit = 4):
        """
        The constructor for the Mancala class defines several instance variables:

        pits_per_player: This variable stores the number of pits each player has.
        stones_per_pit: It represents the number of stones each pit contains at the start of any game.
        board: This data structure is responsible for managing the Mancala board.
        current_player: This variable takes the value 1 or 2, as it's a two-player game, indicating which player's turn it is.
        moves: This is a list used to store the moves made by each player. It's structured in the format (current_player, chosen_pit).
        p1_pits_index: A list containing two elements representing the start and end indices of player 1's pits in the board data structure.
        p2_pits_index: Similar to p1_pits_index, it contains the start and end indices for player 2's pits on the board.
        p1_mancala_index and p2_mancala_index: These variables hold the indices of the Mancala pits on the board for players 1 and 2, respectively.
        """
        self.pits_per_player = pits_per_player
        self.board = [stones_per_pit] * ((pits_per_player+1) * 2)  # Initialize each pit with stones_per_pit number of stones 
        self.players = 2
        self.current_player = 1
        self.moves = []
        self.p1_pits_index = [0, self.pits_per_player-1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player+1, len(self.board)-1-1]
        self.p2_mancala_index = len(self.board)-1
        
        # Zeroing the Mancala for both players
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0

    def display_board(self):
        """
        Displays the board in a user-friendly format
        """
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        player_2_mancala = self.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            else:    
                print('{} -> | {} | {} | <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            
        print('         {}         '.format(player_1_mancala))
        turn = 'P1' if self.current_player == 1 else 'P2'
        print('Turn: ' + turn)
        
    def valid_move(self, pit):
        """
        Function to check if the pit chosen by the current_player is a valid move.
        """
        check = False
        
        # write your code here
        # if the pit is out of range or empty
        if(self.current_player == 1):
            if(pit <= self.pits_per_player):
                if(self.board[pit - 1] != 0):
                    check = True
        else:
            if(pit <= self.pits_per_player):
                if(self.board[pit + self.pits_per_player] != 0):
                    check = True
                    
        return check
        
    def random_move_generator(self):
        """
        Function to generate random valid moves with non-empty pits for the random player
        """
        # write your code here
        valid = False
        
        # generating a random valid move
        while(valid == False):
            move = randint(1, self.pits_per_player)
            valid = self.valid_move(move)
            
        return move
    
    def play(self, pit):
        """
        This function simulates a single move made by a specific player using their selected pit. It primarily performs three tasks:
        1. It checks if the chosen pit is a valid move for the current player. If not, it prints "INVALID MOVE" and takes no action.
        2. It verifies if the game board has already reached a winning state. If so, it prints "GAME OVER" and takes no further action.
        3. After passing the above two checks, it proceeds to distribute the stones according to the specified Mancala rules.

        Finally, the function then switches the current player, allowing the other player to take their turn.
        """
        # write your code here
        # print("Player", self.current_player, "chose pit:", pit)
        # checking if pit is valid
        test = self.valid_move(pit)
        if(test != True):
            print("INVALID MOVE")
            print()
            return self.board
        
        # print()
    
        # checking for winning state
        win = self.winning_eval()
        if(win == (True, 1) or win == (True, 2) or win == (True, -1)):
            # print("GAME OVER")
            return self.board
        
        # determining current player
        if(self.current_player == 1):
            self.moves.append((1, pit))
            moves = self.board[pit - 1]
            self.board[pit - 1] = 0 # adjusting current count
            for i in range(1, moves + 1):
                nextpit = (pit + i - 1) % (len(self.board))
                # make sure that pit is not opponent's mancala
                if(nextpit == self.p2_mancala_index):
                    nextpit = (nextpit + 1) % (len(self.board))
                    
                # if on last move
                if(i == (moves)): 
                    if(self.board[nextpit] == 0): # the next pit is empty
                        # check if it is our side but not our mancala
                        if(nextpit in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1)):
                            # print("Capture!")
                            opp_pit = 2*self.pits_per_player - nextpit
                            extras = self.board[opp_pit] + 1 # taking the current marble as well
                            self.board[opp_pit] = 0
                            # place these in my mancala
                            self.board[self.p1_mancala_index] += extras
                            break

                self.board[nextpit] += 1 # depositing a marble
                        
            # switching to player 2
            self.current_player = 2
        else:
            self.moves.append((2, pit))
            # moving base pit up accordingly to be at the right index
            pit = pit + self.pits_per_player + 1
            moves = self.board[pit - 1]
            self.board[pit - 1] = 0 # adjusting current count
            for i in range(1, moves + 1):
                nextpit = (pit + i - 1) % (len(self.board))
                # make sure that pit is not opponent's mancala
                if(nextpit == self.p1_mancala_index):
                    nextpit = (nextpit + 1) % (len(self.board))
                # if on last move
                if(i == (moves)): 
                    if(self.board[nextpit] == 0): # the next pit is empty
                        # check if it is our side but not our mancala
                        if(nextpit in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1)):
                            # print("Capture!")
                            opp_pit = 2*self.pits_per_player - nextpit
                            extras = self.board[opp_pit] + 1 # taking the current marble as well
                            self.board[opp_pit] = 0
                            # place these in my mancala
                            self.board[self.p2_mancala_index] += extras
                            break
                            
                self.board[nextpit] += 1 # deposting a marble
                        
            # switching to player 1
            self.current_player = 1
                   
        return self.board
    
    def winning_eval(self):
        """
        Function to verify if the game board has reached the winning state.
        Hint: If either of the players' pits are all empty, then it is considered a winning state.
        """
        # if a player's side is empty, the game is over
        # the player who still has stones on their side of the board places these into their mancala
        # whoever has the most stones in their mancala wins
        state = True
        pl = -1
        
        for i in range(self.pits_per_player): # player one's side is empty
            if (self.board[i] != 0):
                state = False
                
        if(state == True):
            # count the number of stones on player two's side, adding these to their mancala
            for i in range(self.pits_per_player):
                stones = self.board[self.pits_per_player + 1 + i]
                self.board[self.pits_per_player + 1 + i] = 0 # taking them out of the pits
                self.board[self.p2_mancala_index] = self.board[self.p2_mancala_index] + stones
                
            # determining who won
            if (self.board[self.p1_mancala_index] > self.board[self.p2_mancala_index]):
                # player one wins
                return (state, 1)
            elif (self.board[self.p2_mancala_index] > self.board[self.p1_mancala_index]):
                # player two wins
                return (state, 2)
            else:
                # tie
                return (state, 0)
    
        state = True
        for i in range(self.pits_per_player): # player two's side is empty
            if (self.board[self.pits_per_player + 1 + i] != 0):
                state = False
                
        if (state == True):
            # count the number of stones on player one's side, adding these to their mancala
            for i in range(self.pits_per_player):
                stones = self.board[i]
                self.board[i] = 0
                self.board[self.p1_mancala_index] = self.board[self.p1_mancala_index] + stones
                
            # determining who won
            if (self.board[self.p1_mancala_index] > self.board[self.p2_mancala_index]):
                # player one wins
                return (state, 1)
            elif (self.board[self.p2_mancala_index] > self.board[self.p1_mancala_index]):
                # player two wins
                return (state, 2)
            else:
                # tie
                return (state, 0)
        
        return (state, pl)

    def utility(self):
        # calculates the utility of playing a given pit for a player
        score_diff_weight = 0.5
        opponent_score_weight = -0.2
        score_weight = 0.7

        # basic scores
        p1_score = self.board[self.p1_mancala_index] - self.board[self.p2_mancala_index]
        p2_score = self.board[self.p2_mancala_index] - self.board[self.p1_mancala_index]
        
        # total number of marbles to move for each player
        p1_tot = sum(self.board[self.p1_pits_index[0]:self.p1_pits_index[1]+1])
        p2_tot = sum(self.board[self.p2_pits_index[0]:self.p2_pits_index[1]+1])
        
        # number of pits with marbles
        p1_fill = sum(1 for i in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1) if self.board[i] > 0)
        p2_fill = sum(1 for i in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1) if self.board[i] > 0)
        
        if (p1_fill == 0 or p2_fill == 0):
            # someone won the game
            return (score_diff_weight * (p1_score - p2_score) + (opponent_score_weight - 0.5) * (p2_tot - 0.5) + score_weight * p1_tot)

        # maximize player one's ratio
        p1_rat = p1_tot/p1_fill
        p2_rat = p2_tot/p2_fill
        
        utility_score = (score_diff_weight * (p1_score - p2_score) + opponent_score_weight * p2_rat + score_weight * p1_rat)
        
        return utility_score
    
    def minimax(self, depth, max_depth, maximizing_player):
        if depth == max_depth or self.winning_eval()[0]:
            return self.utility(), None

        if maximizing_player:  # Player 1 (Max)
            best_value = float('-inf')
            best_move = None
            for pit in range(1, self.pits_per_player + 1):
                if self.valid_move(pit):
                    game_copy = deepcopy(self)
                    game_copy.play(pit)
                    value, _ = game_copy.minimax(depth + 1, max_depth, False)
                    if value > best_value:
                        best_value = value
                        best_move = pit
            return best_value, best_move
        else:  # Player 2 (Min)
            best_value = float('inf')
            best_move = None
            for pit in range(1, self.pits_per_player + 1):
                if self.valid_move(pit):
                    game_copy = deepcopy(self)
                    game_copy.play(pit)
                    value, _ = game_copy.minimax(depth + 1, max_depth, True)
                    if value < best_value:
                        best_value = value
                        best_move = pit
            return best_value, best_move

    def get_best_move(self, max_depth):
        _, move = self.minimax(0, max_depth, self.current_player == 1)
        return move 
    
    def alpha_beta(self, depth, alpha, beta, max_depth, maximizing_player):
        if depth == max_depth or self.winning_eval()[0]:
            return self.utility(), None

        if maximizing_player:  # Player 1 (Max)
            best_value = float('-inf')
            best_move = None
            for pit in range(1, self.pits_per_player + 1):
                if self.valid_move(pit):
                    game_copy = deepcopy(self)
                    game_copy.play(pit)
                    value, _ = game_copy.alpha_beta(depth + 1, alpha, beta, max_depth, False)
                    if value > best_value:
                        best_value = value
                        best_move = pit
                    alpha = max(alpha, best_value)
                    
                    # alpha beta pruning
                    if beta <= alpha:
                        break
                        
            return best_value, best_move
        else:  # Player 2 (Min)
            best_value = float('inf')
            best_move = None
            for pit in range(1, self.pits_per_player + 1):
                if self.valid_move(pit):
                    game_copy = deepcopy(self)
                    game_copy.play(pit)
                    value, _ = game_copy.alpha_beta(depth + 1, alpha, beta, max_depth, True)
                    if value < best_value:
                        best_value = value
                        best_move = pit
                    beta = min(beta, best_value)
                    
                    # alpha beta pruning
                    if beta <= alpha:
                        break
            return best_value, best_move
        
        
    def alpha_beta_move(self, max_depth):
        _, move = self.alpha_beta(0, float('-inf'), float('inf'), max_depth, self.current_player == 1)
        return move

In [ ]:
# Simulating 100 games of AI with AB pruning and improved utility function vs Random player
ai_wins = 0  
random_wins = 0  
ties = 0

# Move counters
ai_moves = 0
random_moves = 0

max_depth = 10  # Number of plies

n = 100

start_time = time.perf_counter()

for i in range(n):
    game = Mancala()
    while(game.winning_eval() == (False, -1)):
        player = game.current_player
        if player == 1:  # AI alpha-beta player
            move = game.alpha_beta_move(max_depth)
            ai_moves += 1
        else:  # Random player
            move = game.random_move_generator()
            random_moves += 1
        game.play(move)
        # game.display_board()
        # print()

    win = game.winning_eval()
    
    if (win == (True, 1)):
        ai_wins += 1
    elif (win == (True, 2)):
        random_wins += 1
    else:
        ties += 1
        
    del game # removing the game for the next round
        
    # print()
    # print()
    
end_time = time.perf_counter()
tot_elapsed_time = end_time - start_time
avg_elapsed_time = tot_elapsed_time/100

print(f"The average amount of time per game is {avg_elapsed_time:.4f} seconds.")
print()

ai_wins = ai_wins/n * 100
random_wins = random_wins/n * 100
ties = ties/n * 100

print(f"AI (Player 1) won {ai_wins}% of the time.")
print(f"Random (Player 2) won {random_wins}% of the time.")
print(f"Ties occurred {ties}% of the time.")
print()

p1avgmoves = ai_moves/n
p2avgmoves = random_moves/n

print("Player 1 won with an average of " + str(p1avgmoves) + " moves per game.")
print("Player 2 won with an average of " + str(p2avgmoves) + " moves per game.")


With ten plies, It takes about 0.4 seconds for a game to run to completion, and AI wins around 99% of the time, with about 13 moves. With only about 0.05 more seconds per game, the AI wins almost 100% of the time in around 2 less moves than the minimax and alpha-beta AI player. Increasing the number of plies increases the success rate of the AI player because it means that the alpha-beta algorithm is going through multiple rounds of the game to see which current action will be most favorable in the long run.

The utility function is better than the one given to us, since we are considering what moves can be made by each player, and not just who has the most amount of marbles in their mancala at any given point in time. Most importantly, it was missing the importance of captures. Even if a player has the most marbles in their mancala with a move, they could be leaving their board open to more captures in following moves by the opponent, leading to the AI making a "bad" move. We improved our utility function by accounting for how many marbles were in each player's mancala, how many marbles were on each player's side of the board, and how many pits were available to take from for any given player. We set different weights for each of these, including a negative weight for the opponent since we don't want to make a move that would benefit the opponent. Our utility function places more value on having more marbles in the pits and more spaces to move from, since this gives us more moves to take but also could allow us to put more marbles in our mancala. We also considered stones already in our mancala, but gave it a lesser weight since this is not what we want to prioritize in our utility function. By using the average of stones per non-empty pit, we can determine a numeric value for how "actionable" we are, giving us more moves to make, and potentially better moves to make. This utility function is a better way to evaluate the strength of a match because it is considering long term scores and actions, rather than immediate success. This is better because it considers the potential a player has to win, and thinks in the long run for moves to make that would produce a win. 